In [ ]:
#ここから　このセルで適切な定数を設定

import os
os.chdir(履歴をインポートし保存するディレクトリを指定)
os.getcwd()

init_dmid = '' #取得したいDMのIDをここで設定　　dmConversation - conversationIdの内容を文字列で

In [ ]:
import json

from dateutil import parser
from pytz import timezone

#ユーザーIDリスト作成
UserIdList=list()

with open('direct-message-group.js',encoding='utf_8') as f:
    tjd=json.load(f)
    
    i=0
    for tjdpart in tjd:
        if not tjdpart['dmConversation']['conversationId'] == init_dmid:
            break
        #print(i)
        #i=i+1
        #print(tjdpart['dmConversation']['conversationId'] )

        for tjdcon in (tjdpart['dmConversation']['messages']):
            try:
                if not tjdcon['messageCreate']['senderId'] in UserIdList:
                    UserIdList.append(tjdcon['messageCreate']['senderId'])
            except KeyError:
                print('KeyError : ')
                print(tjdcon.keys())
                
            

In [ ]:
#Wikiでユーザー名見やすいようユーザー名とIDの辞書を作ろう ユーザー名は手動で調べて入れること

import pickle

FileReadOkFlag = 0
try:
    wf = open('Userlist.bin', mode='rb')
    Userlist = pickle.load(wf)
    print("既存ユーザーリストを読み込みました。登録状況は")
    print(Userlist)
    print("です")
    wf.close()
    FileReadOkFlag = 1
except EOFError:
    print("既存ユーザーリストが空か正常に読み取れませんでした(EOF)")
    
if not FileReadOkFlag == 1:
    print("ユーザーリストを新規作成します")
    Userlist={}
    
wf = open('Userlist.bin', mode='wb')

for Ids in UserIdList:
    if not Ids in Userlist:
        print(Ids + 'は誰ですか？')
        Userlist[Ids] = input()
    
pickle.dump(Userlist, wf)

In [ ]:
#Wikiに貼り付けるDM一覧作成
import datetime

UserIdList=list()
wf = open('WikiWrite.txt',mode='w',encoding='utf_8')

with open('direct-message-group.js',encoding='utf_8') as f:
    tjd=json.load(f)
    init_dmid = tjd[0]['dmConversation']['conversationId'] #取得したいDMのIDをここで設定
    element_count = 0
    keep_date = datetime.datetime(1990, 1, 1, 1, 1, 1)
    samemonth_count = 1
    
    for tjdpart in reversed(tjd):
        if not tjdpart['dmConversation']['conversationId'] == init_dmid:
            continue
        print(tjdpart['dmConversation']['conversationId'] )
        print(tjdpart['dmConversation']['messages'][0].keys() )
        
        for tjdcon in reversed(tjdpart['dmConversation']['messages']):
            element_count = element_count + 1
            fileadd_flag = 0
            if element_count > 500:
                samemonth_count = samemonth_count + 1
                fileadd_flag = 1
            
            if 'messageCreate' in tjdcon:
                jst_time = parser.parse(tjdcon['messageCreate']['createdAt']).astimezone(timezone('Asia/Tokyo'))
                if not keep_date.month == jst_time.month or fileadd_flag == 1:
                    if not fileadd_flag == 1:
                        samemonth_count = 1
                    element_count = 0
                    keep_date = jst_time
                    wf.close()
                    wf = open(jst_time.strftime('%Y-%m_WikiWrite')+str(samemonth_count)+'.txt',mode='w',encoding='utf_8')
                
                tmp_str = tjdcon['messageCreate']['text'].replace('\n','\n　　　　') #改行→　改行＋全角スペースで画面左端にくっつくことを防ぐ
                element_count = element_count + tmp_str.count('\n')
                tmp_str2 = ""
                for c in tmp_str:
                    ord_num = ord(c)
                    if ord_num > 0x1F003 and ord_num < 0x1F9C1:
                        tmp_str2 = tmp_str2 + ' &size(3){ - (絵文字'+hex(ord_num)+')}'
                    elif ord_num > 0x20000 and ord_num < 0x2FFFF:
                        tmp_str2 = tmp_str2 + ' &size(3){ - (特殊文字'+hex(ord_num)+')}'
                    elif ord_num > 0xE0000 and ord_num < 0xEFFFF:
                        tmp_str2 = tmp_str2 + ' &size(3){ - (特殊文字'+hex(ord_num)+')}'
                    else:
                        tmp_str2 = tmp_str2 + c
                        
                wf.write(Userlist[tjdcon['messageCreate']['senderId']]+'\t:\t'+tmp_str2+' &size(5){ - '+jst_time.strftime("%Y/%m/%d %H:%M:%S")+'}\n')
            elif 'participantsJoin' in tjdcon:
                jst_time = parser.parse(tjdcon['participantsJoin']['createdAt']).astimezone(timezone('Asia/Tokyo'))
                if not keep_date.month == jst_time.month or fileadd_flag == 1:
                    if not fileadd_flag == 1:
                        samemonth_count = 1
                    element_count = 0
                    keep_date = jst_time
                    wf.close()
                    wf = open(jst_time.strftime('%Y-%m_WikiWrite')+str(samemonth_count)+'.txt',mode='w',encoding='utf_8')
                    
                for joinedIds in tjdcon['participantsJoin']['userIds']:
                    wf.write('　～～～～　'+Userlist[joinedIds]+'さんが加わりました　～～～～　'+' &size(5){ - '+jst_time.strftime("%Y/%m/%d %H:%M:%S")+'}\n')
                
            elif 'conversationNameUpdate' in tjdcon:
                jst_time = parser.parse(tjdcon['conversationNameUpdate']['createdAt']).astimezone(timezone('Asia/Tokyo'))
                if not keep_date.month == jst_time.month or fileadd_flag == 1:
                    if not fileadd_flag == 1:
                        samemonth_count = 1
                    element_count = 0
                    keep_date = jst_time
                    wf.close()
                    wf = open(jst_time.strftime('%Y-%m_WikiWrite')+str(samemonth_count)+'.txt',mode='w',encoding='utf_8')
                    
                print(type(tjdcon['conversationNameUpdate']['name']))
                wf.write('　～～～～　名前が'+tjdcon['conversationNameUpdate']['name']+' に変更されました　～～～～　'+' &size(5){ - '+jst_time.strftime("%Y/%m/%d %H:%M:%S")+'}\n')
                
            elif 'joinConversation'  in tjdcon:
                jst_time = parser.parse(tjdcon['joinConversation']['createdAt']).astimezone(timezone('Asia/Tokyo'))
                if not keep_date.month == jst_time.month or fileadd_flag == 1:
                    if not fileadd_flag == 1:
                        samemonth_count = 1
                    element_count = 0
                    keep_date = jst_time
                    wf.close()
                    wf = open(jst_time.strftime('%Y-%m_WikiWrite')+str(samemonth_count)+'.txt',mode='w',encoding='utf_8')
                    
                wf.write('　～～～～　このログの観測者が会話に加わりました　～～～～　'+' &size(5){ - '+jst_time.strftime("%Y/%m/%d %H:%M:%S")+'}\n')
wf.close()